# 1. 관련 모듈 불러오기

In [1]:
import nltk
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
import pandas as pd
import re
from konlpy.tag import Mecab 
from gensim.test.utils import common_texts

# 2. 백터화 할 데이터 불러오기

In [2]:
df = pd.DataFrame(pd.read_csv('005_0001467715.txt', sep='\t\t',header = None)) #데이터를 불러옵니다

C:\Users\82109\anaconda3\lib\site-packages\pandas\util\_decorators.py:311: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  return func(*args, **kwargs)


In [3]:
df # 한 문서를 한 행에 저장한 데이터입니다.

,0,1
0,0,"ㅋㅋㅋ 종교인들은 냉동인간, 영생 관련 테크 및 노화방지 연구에 매우 비관적임 왜냐..."
1,1,깨어났다고 해도 100년뒤 아무도없는 세상에서 혼자있는게 더지옥일건데 사망선고 했을...
2,2,보내라
3,3,이미 죽은 사람을 다시 살릴 기술이 100년내에 생길까 싶지만... 돈까지 들여가...
4,4,죽었는데 그걸 고친다고 다시살아날까?
...,...,...
993,993,지금 과학발전 정도를 보면 아마 1000년은 지나야 냉동인간을 깨우는게 가능한지 알...
994,994,그냥 편안하게 보내드리지... 참 뭐하는 짓이냐.. 말도 안되는 일이지만 과학 발...
995,995,공수레 공수거
996,996,좋게보내주는게 최고입니다


In [4]:
df.columns=['index','text'] #컬럼명은 이해할 수 있는 말로 바꿔주는 게 좋아요

# 3. 데이터 전처리 하기

In [5]:
df=df.iloc[:, 1:] #데이터 프레임에서 인덱스는 날리고 텍스트 부분만 남길게요
df# text 내용이 None으로 입력된 행이 있네요 

,text
0,"ㅋㅋㅋ 종교인들은 냉동인간, 영생 관련 테크 및 노화방지 연구에 매우 비관적임 왜냐..."
1,깨어났다고 해도 100년뒤 아무도없는 세상에서 혼자있는게 더지옥일건데 사망선고 했을...
2,보내라
3,이미 죽은 사람을 다시 살릴 기술이 100년내에 생길까 싶지만... 돈까지 들여가...
4,죽었는데 그걸 고친다고 다시살아날까?
...,...
993,지금 과학발전 정도를 보면 아마 1000년은 지나야 냉동인간을 깨우는게 가능한지 알...
994,그냥 편안하게 보내드리지... 참 뭐하는 짓이냐.. 말도 안되는 일이지만 과학 발...
995,공수레 공수거
996,좋게보내주는게 최고입니다


In [6]:
df.info() # 데이터 정보 살펴보기 : NULL 값이 있는지 확인하는 과정입니다.

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 998 entries, 0 to 997
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    794 non-null    object
dtypes: object(1)
memory usage: 7.9+ KB


In [7]:
spac = df[df['text'].str.contains('None',na=True)].index #ext 내용이 None으로 입력된 행만을 저장하고
df.drop(spac, inplace=True) #이를 지울게요
df

C:\Users\82109\anaconda3\lib\site-packages\pandas\core\frame.py:4913: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


,text
0,"ㅋㅋㅋ 종교인들은 냉동인간, 영생 관련 테크 및 노화방지 연구에 매우 비관적임 왜냐..."
1,깨어났다고 해도 100년뒤 아무도없는 세상에서 혼자있는게 더지옥일건데 사망선고 했을...
2,보내라
3,이미 죽은 사람을 다시 살릴 기술이 100년내에 생길까 싶지만... 돈까지 들여가...
4,죽었는데 그걸 고친다고 다시살아날까?
...,...
991,고인이 혹시 먼 미래에 의학발전의 힘을 받아 다시 삶을 살 수 있다해도 과연 행복하...
993,지금 과학발전 정도를 보면 아마 1000년은 지나야 냉동인간을 깨우는게 가능한지 알...
994,그냥 편안하게 보내드리지... 참 뭐하는 짓이냐.. 말도 안되는 일이지만 과학 발...
995,공수레 공수거


In [8]:
df['TextCleaning'] = df['text'].apply(lambda x :re.sub(r'[^\s\w\d]',' ', x)) # 한글만 남기고 기호 등을 지울게요

C:\Users\82109\anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [9]:
mecab = Mecab('C:\mecab\mecab-ko-dic')
df['Token_Description'] = df['TextCleaning'].apply(lambda x : mecab.morphs(x))#데이터를 token화 합니다
df=df.reset_index()
df

C:\Users\82109\anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,index,text,TextCleaning,Token_Description
0,0,"ㅋㅋㅋ 종교인들은 냉동인간, 영생 관련 테크 및 노화방지 연구에 매우 비관적임 왜냐...",ㅋㅋㅋ 종교인들은 냉동인간 영생 관련 테크 및 노화방지 연구에 매우 비관적임 왜냐...,"[ㅋㅋㅋ, 종교인, 들, 은, 냉동, 인간, 영생, 관련, 테크, 및, 노화, 방지..."
1,1,깨어났다고 해도 100년뒤 아무도없는 세상에서 혼자있는게 더지옥일건데 사망선고 했을...,깨어났다고 해도 100년뒤 아무도없는 세상에서 혼자있는게 더지옥일건데 사망선고 했을...,"[깨어났, 다고, 해도, 100, 년, 뒤, 아무, 도, 없, 는, 세상, 에서, ..."
2,2,보내라,보내라,"[보내, 라]"
3,3,이미 죽은 사람을 다시 살릴 기술이 100년내에 생길까 싶지만... 돈까지 들여가...,이미 죽은 사람을 다시 살릴 기술이 100년내에 생길까 싶지만 돈까지 들여가...,"[이미, 죽, 은, 사람, 을, 다시, 살릴, 기술, 이, 100, 년, 내, 에,..."
4,4,죽었는데 그걸 고친다고 다시살아날까?,죽었는데 그걸 고친다고 다시살아날까,"[죽, 었, 는데, 그걸, 고친다고, 다시, 살, 아, 날까]"
...,...,...,...,...
789,991,고인이 혹시 먼 미래에 의학발전의 힘을 받아 다시 삶을 살 수 있다해도 과연 행복하...,고인이 혹시 먼 미래에 의학발전의 힘을 받아 다시 삶을 살 수 있다해도 과연 행복하...,"[고인, 이, 혹시, 먼, 미래, 에, 의학, 발전, 의, 힘, 을, 받, 아, 다..."
790,993,지금 과학발전 정도를 보면 아마 1000년은 지나야 냉동인간을 깨우는게 가능한지 알...,지금 과학발전 정도를 보면 아마 1000년은 지나야 냉동인간을 깨우는게 가능한지 알...,"[지금, 과학, 발전, 정도, 를, 보, 면, 아마, 1000, 년, 은, 지나, ..."
791,994,그냥 편안하게 보내드리지... 참 뭐하는 짓이냐.. 말도 안되는 일이지만 과학 발...,그냥 편안하게 보내드리지 참 뭐하는 짓이냐 말도 안되는 일이지만 과학 발...,"[그냥, 편안, 하, 게, 보내, 드리, 지, 참, 뭐, 하, 는, 짓, 이, 냐,..."
792,995,공수레 공수거,공수레 공수거,"[공, 수레, 공수, 거]"


# 4. Doc2vec

In [10]:
doc_df = df[['index','Token_Description']].values.tolist() # 백터화 과정을 위해 각 문서를 TaggedDocument로 변경해줍니다.
tagged_data = [TaggedDocument(words=_d, tags=[uid]) for uid, _d in doc_df]

In [11]:
tagged_data

[TaggedDocument(words=['ㅋㅋㅋ', '종교인', '들', '은', '냉동', '인간', '영생', '관련', '테크', '및', '노화', '방지', '연구', '에', '매우', '비관', '적', '임', '왜냐면', '죽', '어서', '천', '국가', '고', '죽', '은', '후', '환생', '해야', '하', '거든', '또는', '윤리', '적', '으로', '문제', '가', '있', '다고', '생각', '하', '는', '사람', '들', '도', '태반', '하', '지만', '먼', '미래', '에', '항성', '간', '이동', '및', '다행', '성종', '이', '될', '인류', '에게', '는', '멸종', '을', '방지', '할', '필수', '적', '인', '기술', '임', '인류', '의', '사이보그', '화', '도', '결국', '불가피', '함', '종교', '전쟁', '안', '일어나', '면', '다행', '이', '라고', '생각', '함', 'ㅎ'], tags=[0]),
 TaggedDocument(words=['깨어났', '다고', '해도', '100', '년', '뒤', '아무', '도', '없', '는', '세상', '에서', '혼자', '있', '는', '게', '더', '지옥', '일', '건데', '사망', '선고', '했', '을', '때', '이미', '뇌', '랑', '심장', '이', '다', '죽', '었', '는데', '무의미', '하', '다', '그냥', '1', '억', '짜리', '미이라', '를', '만드', '는', '거', '같', '은데'], tags=[1]),
 TaggedDocument(words=['보내', '라'], tags=[2]),
 TaggedDocument(words=['이미', '죽', '은', '사람', '을', '다시', '살릴', '기술', '이', '100', '년', '내', '에', '생길까', '싶', '지만', '

In [12]:
max_epochs = 10

model = Doc2Vec(
    window=5, # 보통 5~10 사이로 정합니다
    vector_size=100, # 백터의 차원을 정해주세요
    alpha=0.025, 
    min_alpha=0.025,
    min_count=2,
    dm =1,
    negative = 5,
    seed = 9999)
  
model.build_vocab(tagged_data)

for epoch in range(max_epochs):
    print('iteration {0}'.format(epoch))
    model.train(tagged_data,
                total_examples=model.corpus_count,
                epochs=model.epochs)
    # decrease the learning rate
    model.alpha -= 0.002
    # fix the learning rate, no decay
    model.min_alpha = model.alpha

iteration 0
iteration 1
iteration 2
iteration 3
iteration 4
iteration 5
iteration 6
iteration 7
iteration 8
iteration 9


In [13]:
model.random.seed(0)

In [13]:
model.docvecs  #모델 학습이 잘 되었다면 오류 없이 실행이 될거에요

C:\Users\82109\anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `docvecs` (The `docvecs` property has been renamed `dv`.).
  """Entry point for launching an IPython kernel.


# 5. 결과 확인하기

In [14]:
data_df = pd.DataFrame(model.dv.get_normed_vectors()) # Get all embedding vectors normalized to unit L2 length (euclidean), as a 2D numpy array.
data_df

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
0,0.024167,0.031787,-0.049957,-0.073325,0.017486,-0.011319,0.026381,0.100423,-0.074556,-0.091710,...,0.044974,0.003582,-0.083914,-0.054978,0.123834,-0.087734,0.012832,-0.024061,0.190918,-0.141987
1,-0.055756,-0.113937,-0.079092,-0.080284,0.112575,0.058549,-0.221781,0.068220,0.095695,-0.027671,...,-0.044694,-0.066585,-0.031567,0.078759,0.153429,0.120712,-0.052110,-0.020518,-0.092578,0.128832
2,-0.095757,-0.077400,0.096473,-0.102384,-0.033807,-0.040361,-0.030491,0.152508,-0.068921,-0.100223,...,0.091106,-0.068104,-0.008363,0.143110,0.084068,0.001813,-0.104228,-0.049087,-0.015864,0.025710
3,-0.167375,-0.077319,-0.064588,-0.005584,-0.037417,-0.001482,0.028685,-0.107401,0.124607,-0.002975,...,0.157309,-0.161288,-0.137874,0.040434,0.083401,0.118326,0.084976,0.008925,0.052757,-0.008656
4,-0.218551,-0.067580,-0.044837,-0.025691,0.054081,-0.002461,-0.079616,0.090489,0.010281,0.068791,...,0.110611,-0.197924,-0.103505,0.090383,-0.043567,0.136460,-0.219791,-0.028026,-0.008377,0.105593
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
992,-0.046595,-0.127841,0.102338,0.113889,-0.012506,-0.036376,0.013168,-0.127899,0.028259,0.107760,...,-0.146685,-0.166158,0.005400,-0.014433,-0.070587,0.103740,-0.069339,0.111221,0.003134,-0.176352
993,-0.026748,0.022746,-0.174862,-0.091253,0.017866,0.015963,0.029964,0.025455,-0.078344,0.096884,...,0.075298,-0.049425,-0.171639,0.168645,0.040417,-0.041424,0.081415,0.053930,-0.042787,-0.044076
994,-0.003380,0.034356,-0.064265,0.081964,-0.016519,-0.077118,-0.023027,-0.074431,0.008610,-0.174798,...,0.067105,-0.136839,-0.098612,0.007649,0.083689,-0.054607,0.032183,-0.004302,0.084012,0.025332
995,-0.203446,-0.067137,-0.064424,0.024598,-0.033964,-0.077812,0.002214,0.163415,0.055066,-0.164265,...,0.083971,-0.043438,-0.067181,0.048799,0.124481,0.029372,-0.016320,-0.019758,0.144057,-0.126037


In [15]:
df['vector'] = df.Token_Description.apply(lambda x : model.infer_vector(x)) # 원래 데이터에 백터화 된 내용을 행으로 추가해서 사용하셔도 됩니다.


In [16]:
df

,index,text,TextCleaning,Token_Description,vector
0,0,"ㅋㅋㅋ 종교인들은 냉동인간, 영생 관련 테크 및 노화방지 연구에 매우 비관적임 왜냐...",ㅋㅋㅋ 종교인들은 냉동인간 영생 관련 테크 및 노화방지 연구에 매우 비관적임 왜냐...,"[ㅋㅋㅋ, 종교인, 들, 은, 냉동, 인간, 영생, 관련, 테크, 및, 노화, 방지...","[-0.1058297, -0.09043871, -0.06095844, -0.1267..."
1,1,깨어났다고 해도 100년뒤 아무도없는 세상에서 혼자있는게 더지옥일건데 사망선고 했을...,깨어났다고 해도 100년뒤 아무도없는 세상에서 혼자있는게 더지옥일건데 사망선고 했을...,"[깨어났, 다고, 해도, 100, 년, 뒤, 아무, 도, 없, 는, 세상, 에서, ...","[-0.12187933, -0.11958803, -0.06590234, -0.036..."
2,2,보내라,보내라,"[보내, 라]","[-0.025896637, -0.026578255, -0.003294996, -0...."
3,3,이미 죽은 사람을 다시 살릴 기술이 100년내에 생길까 싶지만... 돈까지 들여가...,이미 죽은 사람을 다시 살릴 기술이 100년내에 생길까 싶지만 돈까지 들여가...,"[이미, 죽, 은, 사람, 을, 다시, 살릴, 기술, 이, 100, 년, 내, 에,...","[-0.11384339, -0.111402266, -0.022691408, -0.0..."
4,4,죽었는데 그걸 고친다고 다시살아날까?,죽었는데 그걸 고친다고 다시살아날까,"[죽, 었, 는데, 그걸, 고친다고, 다시, 살, 아, 날까]","[-0.06439556, -0.060118947, -0.03638133, -0.02..."
...,...,...,...,...,...
789,991,고인이 혹시 먼 미래에 의학발전의 힘을 받아 다시 삶을 살 수 있다해도 과연 행복하...,고인이 혹시 먼 미래에 의학발전의 힘을 받아 다시 삶을 살 수 있다해도 과연 행복하...,"[고인, 이, 혹시, 먼, 미래, 에, 의학, 발전, 의, 힘, 을, 받, 아, 다...","[-0.16369414, -0.17718008, -0.032662876, -0.11..."
790,993,지금 과학발전 정도를 보면 아마 1000년은 지나야 냉동인간을 깨우는게 가능한지 알...,지금 과학발전 정도를 보면 아마 1000년은 지나야 냉동인간을 깨우는게 가능한지 알...,"[지금, 과학, 발전, 정도, 를, 보, 면, 아마, 1000, 년, 은, 지나, ...","[-0.0805294, -0.08272727, -0.07947493, -0.0573..."
791,994,그냥 편안하게 보내드리지... 참 뭐하는 짓이냐.. 말도 안되는 일이지만 과학 발...,그냥 편안하게 보내드리지 참 뭐하는 짓이냐 말도 안되는 일이지만 과학 발...,"[그냥, 편안, 하, 게, 보내, 드리, 지, 참, 뭐, 하, 는, 짓, 이, 냐,...","[-0.12508808, -0.124440365, -0.019404694, -0.0..."
792,995,공수레 공수거,공수레 공수거,"[공, 수레, 공수, 거]","[-0.03325257, -0.033773474, -0.0048269783, -0...."
